## Курсовой проект

- Дедлайн - 14 января 23:59
- Целевая метрика precision@5
- Бейзлайн решения - MainRecommender
- Сдаем ссылку на github с решением. На github должен быть файл recommendations.csv (user_id | [rec_1, rec_2, ...] с рекомендациями. rec_i - реальные id item-ов (из retail_train.csv)
- Минимальный скор на Тесте 0.18 (retail_test.csv)

In [1]:
!pip install implicit

     |████████████████████████████████| 1.1 MB 5.3 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
  Created wheel for implicit: filename=implicit-0.4.8-cp37-cp37m-linux_x86_64.whl size=4606605 sha256=6694ce448d117687001d36c1ff2c02c8f1233e2c2f8eb337d384d37ea49ddac2
  Stored in directory: /root/.cache/pip/wheels/88/e6/34/25e73cccbaf1a961154bb562a5f86123b68fdbf40e306073d6
Successfully built implicit


In [2]:
!pip install catboost

     |████████████████████████████████| 76.3 MB 25 kB/s 


### Подключение необходимых библиотек

In [4]:
import pandas as pd

import numpy as np

from scipy.sparse import csr_matrix

from implicit import als

import lightgbm as lgb

from catboost import CatBoostClassifier, Pool

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

from src.metrics import precision_at_k, recall_at_k
from src.utils import prefilter_items
from src.recommenders import MainRecommender

import matplotlib.pyplot as plt
%matplotlib inline

import time

import pickle

import warnings
warnings.filterwarnings('ignore')

In [5]:
%matplotlib inline

In [6]:
warnings.filterwarnings('ignore')

### Чтение датасетов

In [7]:
data = pd.read_csv('/content/drive/MyDrive/Course_project_(RS)/retail_train.csv')
item_features = pd.read_csv('/content/drive/MyDrive/Course_project_(RS)/product.csv')
user_features = pd.read_csv('/content/drive/MyDrive/Course_project_(RS)/hh_demographic.csv')
test = pd.read_csv('/content/drive/MyDrive/Course_project_(RS)/retail_test.csv')

In [8]:
data.head()

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0


In [9]:
item_features.head()

,PRODUCT_ID,MANUFACTURER,DEPARTMENT,BRAND,COMMODITY_DESC,SUB_COMMODITY_DESC,CURR_SIZE_OF_PRODUCT
0,25671,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB
1,26081,2,MISC. TRANS.,National,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,
2,26093,69,PASTRY,Private,BREAD,BREAD:ITALIAN/FRENCH,
3,26190,69,GROCERY,Private,FRUIT - SHELF STABLE,APPLE SAUCE,50 OZ
4,26355,69,GROCERY,Private,COOKIES/CONES,SPECIALTY COOKIES,14 OZ


In [10]:
user_features.head()

,AGE_DESC,MARITAL_STATUS_CODE,INCOME_DESC,HOMEOWNER_DESC,HH_COMP_DESC,HOUSEHOLD_SIZE_DESC,KID_CATEGORY_DESC,household_key
0,65+,A,35-49K,Homeowner,2 Adults No Kids,2,None/Unknown,1
1,45-54,A,50-74K,Homeowner,2 Adults No Kids,2,None/Unknown,7
2,25-34,U,25-34K,Unknown,2 Adults Kids,3,1,8
3,25-34,U,75-99K,Homeowner,2 Adults Kids,4,2,13
4,45-54,B,50-74K,Homeowner,Single Female,1,None/Unknown,16


In [11]:
test.head()

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,1340,41652823310,664,912987,1,8.49,446,0.0,52,96,0.0,0.0
1,588,41652838477,664,1024426,1,6.29,388,0.0,8,96,0.0,0.0
2,2070,41652857291,664,995242,5,9.10,311,-0.6,46,96,0.0,0.0
3,1602,41665647035,664,827939,1,7.99,334,0.0,1741,96,0.0,0.0
4,1602,41665647035,664,927712,1,0.59,334,-0.4,1741,96,0.0,0.0


In [12]:
item_features.columns = [col.lower() for col in item_features.columns]
user_features.rename(columns={'household_key': 'user_id'}, inplace=True)
user_features.columns = [col.lower() for col in user_features.columns]
item_features.rename(columns={'product_id': 'item_id'}, inplace=True)

### Деление данных на тренировочный, валидационный и тестовый датасеты

#### Схема обучения: -- давние покупки -- | -- 6 недель -- | -- 3 недели --

In [13]:
max_week = data['week_no'].max()
condition_train = (data['week_no'] < max_week - 9)
condition_valid = data['week_no'] >= max_week - 3
condition_test = ((data['week_no'] >= max_week - 9) & (data['week_no'] < max_week - 3))
                   
train_L1 = data[condition_train]
test_L1 = data[condition_test]

train_L2 = test_L1.copy()
valid_L2 = data[condition_valid]

### Предварительная фильтрация данных

In [14]:
n_items_before = train_L1['item_id'].nunique()
train_L1 = prefilter_items(train_L1, 
                           item_features=None, 
                           take_n_popular=10000-1)                    
n_items_after = train_L1['item_id'].nunique()
print('Decreased # items from {} to {}'.format(n_items_before, n_items_after))

Decreased # items from 83685 to 10000


### Генерация новых признаков для модели второго уровня

#### Признаки пользователей

##### Перевод категориальных признаков в числовые

In [15]:
user_features['age_desc'].replace(
    {'19-24': 22, '25-34': 30, '35-44': 40, '45-54': 50, '55-64': 60, '65+': 70},
    inplace=True)

user_features['marital_status_code'].replace(
    {'U': 0, 'A': 1, 'B': 2}, inplace=True)

user_features['income_desc'].replace(
    {'Under 15K': 10, '15-24K': 20, '25-34K':30, '35-49K': 40,
     '50-74K': 62, '75-99K': 87, '100-124K': 112, '125-149K': 137, 
     '150-174K': 162, '175-199K': 187, '200-249K': 225, '250K+':375}, inplace=True)

user_features['homeowner_desc'].replace(
    {'Unknown': 0, 'Probable Renter': 1, 'Renter': 2,
     'Probable Owner': 10, 'Homeowner': 20}, inplace=True)

user_features['hh_comp_desc'].replace(
    {'Unknown': 0, 'Single Male': 1, 'Single Female': 2,
     '1 Adult Kids': 4, '2 Adults No Kids': 5, '2 Adults Kids':6},inplace=True)

user_features['household_size_desc'].replace({'5+': 6}, inplace=True) 

user_features['kid_category_desc'].replace(
    {'None/Unknown': 0, '3+': 5}, inplace=True)

user_features.head()

,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc,user_id
0,70,1,40,20,5,2,0,1
1,50,1,62,20,5,2,0,7
2,30,0,30,0,6,3,1,8
3,30,0,87,20,6,4,2,13
4,50,2,62,20,2,1,0,16


#### Признаки товаров

In [16]:
item_features['department'] = item_features['department'].astype('category')

counts = item_features['department'].value_counts()
others = counts[counts < 50].index

label = 'OTHER'

item_features['department'] = item_features['department'].cat.add_categories([label])
item_features['department'] = item_features['department'].replace(others, label)

item_features['commodity_desc'] = item_features['commodity_desc'].astype('category')

counts = item_features['commodity_desc'].value_counts()
others = counts[counts < 50].index

label = 'OTHER'

item_features['commodity_desc'] = item_features['commodity_desc'].cat.add_categories([label])
item_features['commodity_desc'] = item_features['commodity_desc'].replace(others, label)

commodities = item_features.commodity_desc.value_counts()
commodities_list = commodities.keys().tolist()
for i, name in enumerate(commodities_list):
    item_features.loc[item_features['commodity_desc'] == name, 'commodity_category'] = i
    
item_features['brand'] = np.where(item_features['brand']=='Private', 0, 1)

item_features.head()

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product,commodity_category
0,25671,2,GROCERY,1,OTHER,ICE - CRUSHED/CUBED,22 LB,12.0
1,26081,2,MISC. TRANS.,1,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,,57.0
2,26093,69,PASTRY,0,BREAD,BREAD:ITALIAN/FRENCH,,92.0
3,26190,69,GROCERY,0,FRUIT - SHELF STABLE,APPLE SAUCE,50 OZ,74.0
4,26355,69,GROCERY,0,COOKIES/CONES,SPECIALTY COOKIES,14 OZ,17.0


#### Признаки товаров пользователей и товаров

In [17]:
def get_user_item_features(data_train_lvl_1):
    # час совершения транзакции
    X = data_train_lvl_1.copy()
    X['hour'] = X['trans_time'] // 100
    user_item_features = X.groupby(['user_id', 'item_id'])['hour'].median().reset_index()
    user_item_features.columns = ['user_id', 'item_id', 'median_sales_hour']
    
    # день недели совершения транзакции
    X['weekday'] = X['day'] % 7
    df = X.groupby(['user_id', 'item_id'])['weekday'].median().reset_index()
    df.columns = ['user_id', 'item_id', 'median_weekday']
    user_item_features = user_item_features.merge(df, on=['user_id', 'item_id'])
    
    # средний чек корзины клиента
    df = X.groupby(['user_id', 'basket_id'])['sales_value'].sum().reset_index()
    df = df.groupby('user_id')['sales_value'].mean().reset_index()
    df.columns = ['user_id', 'mean_check']
    user_item_features = user_item_features.merge(df, on=['user_id'])
    
    # количество магазинов, в которых продавался товар
    df = X.groupby(['item_id'])['store_id'].nunique().reset_index()
    df.columns = ['item_id', 'n_stores']
    user_item_features = user_item_features.merge(df, on=['item_id'])
    
    # количество уникальных товаров, купленных клиентом
    df = X.groupby(['user_id'])['item_id'].nunique().reset_index()
    df.columns = ['user_id', 'n_items']
    user_item_features = user_item_features.merge(df, on=['user_id'])
    
    # количество транзакций клиента
    df = X.groupby(['user_id'])['item_id'].count().reset_index()
    df.columns = ['user_id', 'n_transactions']
    user_item_features = user_item_features.merge(df, on=['user_id'])

    # эмбеддинги товаров
    recommender = MainRecommender(X)
    df = recommender.model.item_factors
    n_factors = recommender.model.factors
    ind = list(recommender.id_to_itemid.values())
    df = pd.DataFrame(df, index=ind).reset_index()
    df.columns = ['item_id'] + ['item_f_' + str(i + 1) for i in range(n_factors)]
    user_item_features = user_item_features.merge(df, on=['item_id'])
    
    # эмбеддинги пользователей
    df = recommender.model.user_factors
    ind = list(recommender.id_to_userid.values())
    df = pd.DataFrame(df, index=ind).reset_index()
    df.columns = ['user_id'] + ['user_f_' + str(i + 1) for i in range(n_factors)]
    user_item_features = user_item_features.merge(df, on=['user_id'])
    
    return user_item_features

In [18]:
%%time

user_item_features = get_user_item_features(train_L1)
user_item_features.head()

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/10000 [00:00<?, ?it/s]

CPU times: user 5min 8s, sys: 14min 8s, total: 19min 16s
Wall time: 9min 52s


### Подготовка данных для модели второго уровня

In [19]:
def get_candidates(data_train_lvl_1, 
                   data_train_lvl_2, 
                   N, 
                   add_to_lvl_2):
    recommender = MainRecommender(data_train_lvl_1)

    users_lvl_1 = data_train_lvl_1['user_id'].unique()
    users_lvl_2 = data_train_lvl_2['user_id'].unique().tolist()
    if add_to_lvl_2:
        users_lvl_2 += add_to_lvl_2

    current_users = list(set(users_lvl_2) & set(users_lvl_1))    
    new_users = list(set(users_lvl_2) - set(users_lvl_1))

    df = pd.DataFrame(users_lvl_2, columns=['user_id'])
    cond_1 = df['user_id'].isin(current_users)
    df.loc[cond_1, 'candidates'] = df.loc[cond_1, 'user_id'].apply(
        lambda x: recommender.get_own_recommendations(x, N))

    if new_users:
        cond_2 = df['user_id'].isin(new_users)
        df.loc[cond_2, 'candidates'] = df.loc[cond_2, 'user_id'].apply(
            lambda x: recommender.overall_top_purchases[:N])
        
    return df

In [20]:
def get_targets_lvl_2(data_train_lvl_1, 
                      data_train_lvl_2, 
                      user_item_features, 
                      N, 
                      add_to_lvl_2=None):
    
    users_lvl_2 = get_candidates(data_train_lvl_1, 
                                 data_train_lvl_2, 
                                 N, 
                                 add_to_lvl_2)
    
    df = pd.DataFrame({'user_id': users_lvl_2['user_id'].values.repeat(N),
                       'item_id': np.concatenate(users_lvl_2['candidates'].values)})

    targets_lvl_2 = data_train_lvl_2[['user_id', 'item_id']].copy()
    targets_lvl_2['target'] = 1 

    targets_lvl_2 = df.merge(targets_lvl_2, on=['user_id', 'item_id'], how='left')
    targets_lvl_2['target'].fillna(0, inplace= True)
    
    targets_lvl_2 = targets_lvl_2.merge(item_features, on='item_id', how='left')
    targets_lvl_2 = targets_lvl_2.merge(user_features, on='user_id', how='left')
    targets_lvl_2 = targets_lvl_2.merge(
        user_item_features, on=['user_id', 'item_id'], how='left')
    
    return targets_lvl_2

In [21]:
users_train_L1 = train_L1.user_id.unique()
users_test_L1 = test_L1.user_id.unique()
users_valid_L2 = valid_L2.user_id.unique()

new_users_test_to_train = list(set(users_test_L1) - set(users_train_L1))

new_users_L1_to_valid = list(set(users_valid_L2) - (set(users_train_L1) | set(users_test_L1)))

add_to_lvl_2 = list(set(users_valid_L2) - (set(users_test_L1)))

new_users_test_to_train, new_users_L1_to_valid, len(add_to_lvl_2)

([296, 1813, 1984], [2259], 126)

In [22]:
%%time

N = 1000

targets_lvl_2 = get_targets_lvl_2(train_L1, 
                                  train_L2, 
                                  user_item_features, 
                                  N, 
                                  add_to_lvl_2)

print(f'Число пользователей: {targets_lvl_2.user_id.nunique()}')

targets_lvl_2.head()

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/10000 [00:00<?, ?it/s]

Число пользователей: 2280
CPU times: user 5min 18s, sys: 13min 49s, total: 19min 7s
Wall time: 9min 55s


#### Отбор признаков

In [23]:
SELECTED_FEATURES_NAMES = ['brand',
                           'commodity_category',
                           'age_desc', 
                           'income_desc', 
                           'homeowner_desc', 
                           'hh_comp_desc',
                           'marital_status_code', 
                           'manufacturer',                            
                           'median_sales_hour', 
                           'median_weekday', 
                           'mean_check', 
                           'n_stores', 
                           'n_items', 
                           'n_transactions', 
                           'item_f_1', 
                           'item_f_2', 
                           'item_f_3', 
                           'item_f_4', 
                           'item_f_5',
                           'user_f_1', 
                           'user_f_2', 
                           'user_f_3', 
                           'user_f_4',
                           'user_f_5',
                          ]
categorical = ['marital_status_code',
               'homeowner_desc', 
               'hh_comp_desc', 
               'manufacturer',
               'commodity_category'
              ]

### Подготовка моделей

In [24]:
def run_model_lgb(targets_lvl_2):
    X_train, X_valid, y_train, y_valid = train_test_split(targets_lvl_2[SELECTED_FEATURES_NAMES].fillna(0),
                                                          targets_lvl_2[['target']],
                                                          test_size=0.2, random_state=2021,
                                                          stratify=targets_lvl_2[['target']])

    dtrain = lgb.Dataset(X_train, y_train, categorical_feature=categorical)
    dvalid = lgb.Dataset(X_valid, y_valid, categorical_feature=categorical)

    params_lgb = {"boosting_type": "gbdt",
                  "objective": "binary", 
                  "metric": "auc",
                  "num_boost_round": 10000,
                  "learning_rate": 0.1,
                  "num_leaves": 30,
                  "max_depth": 10,
                  "n_estimators": 5000,
                  "n_jobs": 6,
                  "seed": 2021} 

    model_lgb = lgb.train(params=params_lgb,
                          train_set=dtrain,  
                          valid_sets=[dtrain, dvalid],
                          categorical_feature=categorical,
                          verbose_eval=1000,
                          early_stopping_rounds=10)
    
    return model_lgb

In [25]:
%%time

model_lgb = run_model_lgb(targets_lvl_2)

Training until validation scores don't improve for 10 rounds.
Early stopping, best iteration is:
[619]	training's auc: 0.924609	valid_1's auc: 0.906461
CPU times: user 9min 22s, sys: 8 s, total: 9min 30s
Wall time: 5min 32s


In [26]:
SELECTED_FEATURES_NAMES_cb = [i for i in SELECTED_FEATURES_NAMES if not i in categorical]

def run_model_cb(targets_lvl_2):    
    X_train, X_valid, y_train, y_valid = train_test_split(targets_lvl_2[SELECTED_FEATURES_NAMES_cb].fillna(0),
                                                          targets_lvl_2[['target']],
                                                          test_size=0.2, random_state=2021,
                                                          stratify=targets_lvl_2[['target']])

    dtrain = Pool(data=X_train, label=y_train)
    dvalid = Pool(data=X_valid, label=y_valid)

    params_cb = {"n_estimators":5000,
                 "learning_rate": 0.1,
                 "loss_function": "Logloss",
                 "eval_metric": "AUC",
                 "task_type": "CPU",
                 "max_bin": 30,
                 "early_stopping_rounds": 30,
                 "verbose": 100,
                 "max_depth": 10,
                 "l2_leaf_reg": 80,
                 "thread_count": 6,
                 "random_seed": 2021,
                 } 

    model_cb = CatBoostClassifier(**params_cb)
    
    model_cb.fit(dtrain, eval_set=[dvalid],verbose_eval=False)

    return model_cb

In [27]:
%%time

model_cb = run_model_cb(targets_lvl_2)

CPU times: user 38min 59s, sys: 7.24 s, total: 39min 6s
Wall time: 20min 31s


In [28]:
predictions_lgb_train = model_lgb.predict(targets_lvl_2[SELECTED_FEATURES_NAMES].fillna(0))
predictions_cb_train = model_cb.predict_proba(targets_lvl_2[SELECTED_FEATURES_NAMES_cb].fillna(0))[:, 1]

#### Объединение прогнозов двух моделей

In [29]:
preds_train = pd.DataFrame(zip(predictions_lgb_train, 
                               predictions_cb_train),
                           columns=['lgb', 'cb']).mean(axis=1).values
roc_auc_score(targets_lvl_2['target'], preds_train)

0.9194289760693147

### Обработка прогнозов

In [30]:
def get_predictions(targets_lvl_2, raw_predictions, prefix='lgb'): 
    df = targets_lvl_2[['user_id', 'item_id']]
    df['predictions'] = raw_predictions

    df = df.groupby(['user_id', 'item_id'])['predictions'].median().reset_index()
    df = df.sort_values(['predictions'], ascending=False).groupby(['user_id']).head(5)

    df = df.groupby('user_id')['item_id'].unique().reset_index()
    df.columns = ['user_id', prefix + '_recommendations']
    
    return df

def get_results(data_val_lvl_2, targets_lvl_2, preds_lgb, preds_cb, combined_preds):
    result = data_val_lvl_2.groupby('user_id')['item_id'].unique().reset_index()
    result.columns=['user_id', 'actual']
    
    prefixes = ['lgb', 'cb', 'cb_lgb']
    predictions = [preds_lgb, preds_cb, combined_preds]
    
    for i, preds in enumerate(predictions):
        df = get_predictions(targets_lvl_2, preds, prefixes[i])
        result = result.merge(df, on='user_id', how='left')

    return result

In [31]:
result_lvl_2 = get_results(valid_L2, 
                           targets_lvl_2, 
                           predictions_lgb_train,
                           predictions_cb_train, 
                           preds_train)
result_lvl_2.head()

,user_id,actual,lgb_recommendations,cb_recommendations,cb_lgb_recommendations
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[1082185, 1087895, 922281, 940947, 1029743]","[9527558, 940947, 6391068, 872137, 865456]","[940947, 9527558, 872137, 1087895, 6391068]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[1106523, 1082185, 983584, 1042544, 899624]","[1106523, 5569230, 983584, 899624, 9296919]","[1106523, 983584, 1082185, 5569230, 899624]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[1082185, 1029743, 1070820, 1126899, 1098844]","[1082185, 1098844, 900802, 1029743, 845208]","[1082185, 1098844, 1029743, 1070820, 900802]"
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[1082185, 1106523, 1029743, 1126899, 1122358]","[1106523, 1126899, 1122358, 993638, 12524016]","[1106523, 1126899, 1029743, 1122358, 1082185]"
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[1106523, 1029743, 1070820, 6533765, 1082185]","[1029743, 872137, 1106523, 899624, 1004906]","[1029743, 1106523, 872137, 1041796, 899624]"


### Расчет метрик

#### LightGBM

In [32]:
result_lvl_2.apply(lambda row: precision_at_k(row['lgb_recommendations'], 
                                              row['actual'], 
                                              5), axis=1).mean()

0.273849167482858

#### CatBoost

In [33]:
result_lvl_2.apply(lambda row: precision_at_k(row['cb_recommendations'], 
                                              row['actual'], 
                                              5), axis=1).mean()

0.269343780607246

#### Ансамбль моделей

In [34]:
result_lvl_2.apply(lambda row: precision_at_k(row['cb_lgb_recommendations'], 
                                              row['actual'], 
                                              5), axis=1).mean()

0.28609206660137004

### Построение финальных прогнозов

In [35]:
validation_weeks = 6
data_train = data[data['week_no'] < data['week_no'].max() - validation_weeks]
data_valid = data[data['week_no'] >= data['week_no'].max() - validation_weeks]

In [36]:
users_lvl_1 = data_train.user_id.unique()
users_lvl_2 = data_valid.user_id.unique()
users_lvl_3 = test.user_id.unique()

new_users_lvl_2 = list(set(users_lvl_2) - set(users_lvl_1))
new_users_lvl_3 = list(set(users_lvl_3) - (set(users_lvl_1) | set(users_lvl_2)))

add_to_lvl_2 = list(set(users_lvl_3) - (set(users_lvl_2)))

new_users_lvl_2, new_users_lvl_3, len(add_to_lvl_2)

([], [2325], 75)

In [37]:
n_items_before = data['item_id'].nunique()
data_train = prefilter_items(data_train, 
                           item_features=None, 
                           take_n_popular=10000-1)

n_items_after = data_train['item_id'].nunique()
print('Decreased # items from {} to {}'.format(n_items_before, n_items_after))

Decreased # items from 89051 to 10000


In [38]:
user_item_features = get_user_item_features(data_train)
user_item_features.head()

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/10000 [00:00<?, ?it/s]

,user_id,item_id,median_sales_hour,median_weekday,mean_check,n_stores,n_items,n_transactions,item_f_1,item_f_2,item_f_3,item_f_4,item_f_5,item_f_6,item_f_7,item_f_8,item_f_9,item_f_10,item_f_11,item_f_12,item_f_13,item_f_14,item_f_15,item_f_16,item_f_17,item_f_18,item_f_19,item_f_20,user_f_1,user_f_2,user_f_3,user_f_4,user_f_5,user_f_6,user_f_7,user_f_8,user_f_9,user_f_10,user_f_11,user_f_12,user_f_13,user_f_14,user_f_15,user_f_16,user_f_17,user_f_18,user_f_19,user_f_20
0,1,823721,13.0,4.0,36.641061,95,247,704,0.013754,0.013026,0.004641,0.014826,0.008711,0.007801,0.007825,0.007739,0.016070,0.008426,0.013605,0.007550,0.007097,0.011550,0.016951,0.010348,0.006137,0.005398,0.001238,0.001450,11.156901,8.344952,5.688489,6.395328,-0.46914,2.907827,-3.081624,8.151766,-8.076731,3.874395,8.807744,5.461666,6.950864,0.616627,2.202852,-3.030563,-8.046268,-1.611399,-14.471581,-4.157176
1,1,823990,15.0,6.0,36.641061,95,247,704,0.008155,0.003240,0.011237,0.001043,-0.005018,0.026015,0.000269,0.008513,0.020813,0.007103,0.011307,0.015837,0.004289,0.008921,-0.000203,0.003598,0.016792,0.005283,0.004770,0.001970,11.156901,8.344952,5.688489,6.395328,-0.46914,2.907827,-3.081624,8.151766,-8.076731,3.874395,8.807744,5.461666,6.950864,0.616627,2.202852,-3.030563,-8.046268,-1.611399,-14.471581,-4.157176
2,1,825123,17.0,3.0,36.641061,29,247,704,0.004363,0.014097,0.013748,0.008168,-0.002811,0.008248,-0.002322,0.008214,0.006224,-0.003358,0.004655,-0.012346,0.019974,0.008810,0.006319,0.015330,-0.000857,0.007031,-0.005027,-0.001749,11.156901,8.344952,5.688489,6.395328,-0.46914,2.907827,-3.081624,8.151766,-8.076731,3.874395,8.807744,5.461666,6.950864,0.616627,2.202852,-3.030563,-8.046268,-1.611399,-14.471581,-4.157176
3,1,826695,15.0,1.5,36.641061,31,247,704,-0.003376,0.006094,0.000909,0.004675,0.013424,0.007391,-0.008489,0.006605,0.006269,0.005572,0.013790,0.015920,-0.005298,0.009664,0.013913,0.002185,-0.004753,0.002073,0.006139,-0.000349,11.156901,8.344952,5.688489,6.395328,-0.46914,2.907827,-3.081624,8.151766,-8.076731,3.874395,8.807744,5.461666,6.950864,0.616627,2.202852,-3.030563,-8.046268,-1.611399,-14.471581,-4.157176
4,1,827656,19.0,3.0,36.641061,87,247,704,0.010645,0.005609,0.004886,0.012950,0.002402,0.005574,0.011864,-0.006711,0.009445,0.017507,0.009498,0.002854,0.007726,0.018218,0.015072,0.003757,0.010424,0.011613,-0.008954,0.007750,11.156901,8.344952,5.688489,6.395328,-0.46914,2.907827,-3.081624,8.151766,-8.076731,3.874395,8.807744,5.461666,6.950864,0.616627,2.202852,-3.030563,-8.046268,-1.611399,-14.471581,-4.157176


In [39]:
targets_test = get_targets_lvl_2(data_train, 
                                 data_valid, 
                                 user_item_features, 
                                 N, 
                                 add_to_lvl_2)

print(f'Число пользователей: {targets_test.user_id.nunique()}')
print(f'Среднее число покупок: {round(targets_test["target"].mean(), 4)}')

targets_test.head()

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/10000 [00:00<?, ?it/s]

Число пользователей: 2272
Среднее число покупок: 0.028


,user_id,item_id,target,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product,commodity_category,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc,median_sales_hour,median_weekday,mean_check,n_stores,n_items,n_transactions,item_f_1,item_f_2,item_f_3,item_f_4,item_f_5,item_f_6,item_f_7,item_f_8,item_f_9,item_f_10,item_f_11,item_f_12,item_f_13,item_f_14,item_f_15,item_f_16,item_f_17,item_f_18,item_f_19,item_f_20,user_f_1,user_f_2,user_f_3,user_f_4,user_f_5,user_f_6,user_f_7,user_f_8,user_f_9,user_f_10,user_f_11,user_f_12,user_f_13,user_f_14,user_f_15,user_f_16,user_f_17,user_f_18,user_f_19,user_f_20
0,84,903529,1.0,539,DRUG GM,1,CIGARETTES,CIGARETTES,972976 PK,77.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,2.0,16.506667,15.0,86.0,163.0,0.000745,0.003325,0.019860,0.004508,-0.011429,-0.017014,0.014514,0.005844,0.011901,0.000852,0.005346,0.004040,-0.007316,0.005712,0.002064,0.019957,0.002935,-0.002759,0.004822,-0.011652,5.009583,-0.190487,3.133904,1.448137,-4.242996,-7.161358,1.22381,-2.099301,1.576952,2.335679,-1.877053,3.305793,-1.051182,1.279063,0.445951,6.360059,-0.542704,1.123345,5.25176,-3.300395
1,84,903529,1.0,539,DRUG GM,1,CIGARETTES,CIGARETTES,972976 PK,77.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,2.0,16.506667,15.0,86.0,163.0,0.000745,0.003325,0.019860,0.004508,-0.011429,-0.017014,0.014514,0.005844,0.011901,0.000852,0.005346,0.004040,-0.007316,0.005712,0.002064,0.019957,0.002935,-0.002759,0.004822,-0.011652,5.009583,-0.190487,3.133904,1.448137,-4.242996,-7.161358,1.22381,-2.099301,1.576952,2.335679,-1.877053,3.305793,-1.051182,1.279063,0.445951,6.360059,-0.542704,1.123345,5.25176,-3.300395
2,84,920025,1.0,764,GROCERY,1,LAUNDRY ADDITIVES,FABRIC SOFTENER LIQUID,60 LOAD,131.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,5.0,16.506667,20.0,86.0,163.0,0.012126,-0.004676,0.015723,0.007664,0.009080,-0.014034,0.003026,-0.000333,0.004948,0.009806,0.000502,0.002504,0.006359,0.002745,-0.000446,0.004849,-0.003484,-0.000984,0.011034,-0.000976,5.009583,-0.190487,3.133904,1.448137,-4.242996,-7.161358,1.22381,-2.099301,1.576952,2.335679,-1.877053,3.305793,-1.051182,1.279063,0.445951,6.360059,-0.542704,1.123345,5.25176,-3.300395
3,84,977374,0.0,111,DRUG GM,1,CIGARETTES,CIGARETTES,CTN,77.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,84,884643,0.0,5031,DRUG GM,1,SINUS AND ALLERGY,SINUS AND ALLERGY,,144.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


##### Генерация прогнозов

In [40]:
model_cb = run_model_cb(targets_test)

In [41]:
predictions_lgb_test = model_lgb.predict(targets_test[SELECTED_FEATURES_NAMES].fillna(0))
predictions_cb_test = model_cb.predict_proba(targets_test[SELECTED_FEATURES_NAMES_cb].fillna(0))[:, 1]

In [ ]:
preds_test = pd.DataFrame(zip(predictions_lgb_test, predictions_cb_test),
                          columns=['lgb', 'cb']).mean(axis=1).values
roc_auc_score(targets_test['target'], preds_test)

In [43]:
result_test = get_results(test, targets_test, 
                           predictions_lgb_test,
                           predictions_cb_test, 
                           preds_test)
result_test.head()

,user_id,actual,lgb_recommendations,cb_recommendations,cb_lgb_recommendations
0,1,"[880007, 883616, 931136, 938004, 940947, 94726...","[1082185, 1029743, 1087895, 968164, 1082212]","[1005186, 856942, 8293439, 940947, 9655212]","[940947, 856942, 9655212, 1005186, 1087895]"
1,2,"[820165, 820291, 826784, 826835, 829009, 85784...","[1106523, 1082185, 1068719, 1075368, 916122]","[1075368, 899624, 5569230, 1106523, 8090521]","[1106523, 1075368, 1068719, 899624, 916122]"
2,3,"[827683, 908531, 989069, 1071377, 1080155, 109...","[1082185, 983584, 1106523, 870547, 910032]","[1106523, 5569230, 893018, 983584, 1101010]","[1106523, 983584, 1082185, 870547, 893018]"
3,6,"[956902, 960791, 1037863, 1119051, 1137688, 84...","[1029743, 1070820, 1126899, 1082185, 845208]","[1082185, 5569230, 1098844, 1070820, 1024306]","[1082185, 1029743, 1070820, 1126899, 845208]"
4,7,"[847270, 855557, 859987, 863407, 895454, 90663...","[1082185, 1029743, 1126899, 1106523, 993838]","[1106523, 1126899, 1029743, 1122358, 866211]","[1106523, 1126899, 1029743, 1082185, 1122358]"


In [44]:
result_test.apply(lambda row: precision_at_k(row['lgb_recommendations'],
                                             row['actual'],
                                             5), axis=1).mean()

0.18885941644562088

In [45]:
result_test.apply(lambda row: precision_at_k(row['cb_recommendations'], 
                                             row['actual'], 
                                             5), axis=1).mean()

0.23405835543766348

In [46]:
result_test.apply(lambda row: precision_at_k(row['cb_lgb_recommendations'], 
                                             row['actual'], 
                                             5), axis=1).mean()

0.23427055702917493

### Запись результата в файл

In [48]:
df = result_test[['user_id', 'cb_lgb_recommendations']].copy()
df.to_csv('recommendations.csv', index=False)

df.head()

,user_id,cb_lgb_recommendations
0,1,"[940947, 856942, 9655212, 1005186, 1087895]"
1,2,"[1106523, 1075368, 1068719, 899624, 916122]"
2,3,"[1106523, 983584, 1082185, 870547, 893018]"
3,6,"[1082185, 1029743, 1070820, 1126899, 845208]"
4,7,"[1106523, 1126899, 1029743, 1082185, 1122358]"
